### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2022 Semester 1

## Assignment 1: Naive Bayes Leaner for Adult Database


**Student Name(s):** `PLEASE ENTER YOUR NAME(S) HERE`
<br>
**Student ID(s):** `PLEASE ENTER YOUR ID(S) HERE`



Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

## General info

<b>Due date</b>: Friday, 8 April 2022 7pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: This iPython notebook is a template which you will use for your Assignment 1 submission. You need to only submitted the completed copy of this iPython notebook.

<b>Late submissions</b>: -10% per day up to 5 days (both weekdays and weekends count). Submissions more than 5 days late will not be accepted (resul in a mark of 0).
<ul>
    <li>one day late, -1.0;</li>
    <li>two days late, -2.0;</li>
    <li>three days late, -3.0;</li>
    <li>four days late, -4.0;</li>
    <li>five days late, -5.0;</li>
</ul>

<b>Extensions</b>: Students who are demonstrably unable to submit a full solution in time due to medical reasons or other trauma, may apply for an extension.  In these cases, you should email <a href="mailto:ni.ding@unimelb.edu.au">Ni Ding</a> as soon as possible after those circumstances arise. If you attend a GP or other health care service as a result of illness, be sure to provide a Health Professional Report (HPR) form (get it from the Special Consideration section of the Student Portal), you will need this form to be filled out if your illness develops into something that later requires a Special Consideration application to be lodged. You should scan the HPR form and send it with the extension requests.

<b>Marks</b>: This assignment will be marked out of 20, and make up 20% of your overall mark for this subject.

<b>Materials</b>: Use Jupyter Notebook and Python page on Canvas for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn. You can use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 2 marks for unreadable or exessively inefficient code.

8 of the marks available for this Project will be assigned to whether the four specified Python functions work in a manner consistent with the materials from COMP30027. Any other implementation will not be directly assessed (except insofar as it is required to make these five functions work correctly).

12 of the marks will be assigned to your responses to the questions, in terms of both accuracy and insightfulness. We will be looking for evidence that you have an implementation that allows you to explore the problem, but also that you have thought deeply about the data and the behaviour of the Naive Bayes classifier.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (ED -> Assignments -> A1); we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/124196/modules#module_662096">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

**IMPORTANT**

Please carefully read and fill out the <b>Authorship Declaration</b> form at the bottom of the page. Failure to fill out this form results in the following deductions: 
<UL TYPE=”square”>
<LI>missing Authorship Declaration at the bottom of the page, -5.0
<LI>incomplete or unsigned Authorship Declaration at the bottom of the page, -3.0
</UL>
**NOTE: COMPLETE AND SUBMIT THIS FILE. YOU SHOULD IMPLEMENT FOUR FUNCTIONS AND INCLUDE YOUR ANSWERS TO THE QUESTIONS IN THIS FILE ONLY. NO OTHER SUBMISSION IS REQUIRED.**

**Keep your code clean. Adding proper comments to your code is MANDATORY.**

## Part 1: Base code [8 marks]

Instructions
1. Do **not** shuffle the data set
2. Treat the attributes as they are(e.g., do **not** convert numeric attributes to categorical or categorical to numeric). Implement a Naive Bayes classifier with appropriate likelihood function for each attribute.
3. You should implement the Naive Bayes classifier from scratch. Do **not** use existing implementations/learning algorithms.
4. You CANNOT have more than one train or predict function. Both continuous numeric attributes and categorical ones should be trained in one `train()` function, similarly for the `predict()`.  
5. Apart from the instructions in point 3, you may use libraries to help you with data reading, representation, maths or evaluation
6. Ensure that all and only required information is printed, as indicated in the final three code cells. Failure to adhere to print the required information will result in **[-1 mark]** per case. *(We don't mind details like you print a list or several numbers -- just make sure the information is displayed so that it's easily accessible)
7. You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 
8. You should add adequate comments to make your code easily comprehendible.*

In [190]:
class NaiveBayes:
    # priors = {}
    # likelihood_model = None
    class ConditionalProbability:
        parameters_multinomial = {}
        parameters_gaussian = {}
        def __init__(self, x, y):
            x.apply(lambda c: self.fit_parameter(c, y, c.name), axis=0)
        
        def fit_parameter(self, x_feat, y, feat_name):
            # remove all entries where either feature or label are nan
            nan_indexes = list(set([i for i in range(0, len(x_feat)) if x_feat[i] == np.nan] + [i for i in range(0, len(y)) if y[i] == np.nan]))
            filtered_x_feat = x_feat.drop(nan_indexes)
            filtered_y = y.drop(nan_indexes)
            
            if(x_feat.dtype == "object"):
                self.fit_parameter_categorical(filtered_x_feat, filtered_y, feat_name)
            elif(x_feat.dtype == "int64"):
                self.fit_parameter_continuous(filtered_x_feat, filtered_y, feat_name)
        
        def fit_parameter_continuous(self, x_feat, y, feat_name):
            self.parameters_gaussian[feat_name] = {}
            for y_val in y.unique():
                # todo: check indexing still works after nan removal - maybe cast to list before this
                filtered_x_feat = np.array([x_feat[i] for i in range(0, len(x_feat)) if y[i] == y_val])
                mean = np.mean(filtered_x_feat)
                sd = np.sqrt(np.var(filtered_x_feat, ddof=1))
                self.parameters_gaussian[feat_name][y_val] = {"mean": mean, "sd": sd}
        
        # todo: add smoothing here or in get conditional_probability_categorical
        def fit_parameter_categorical(self, x_feat, y, feat_name):
            self.parameters_multinomial[feat_name] = {}
            for y_val in y.unique():
                self.parameters_multinomial[feat_name][y_val] = {}
                for x_val in x_feat.unique(): 
                    # todo: check indexing still works after nan removal - maybe cast to list before this
                    occurences = [x_feat[i] for i in range(0, len(x_feat)) if y[i] == y_val].count(x_val)
                    total = y.value_counts()[y_val]
                    self.parameters_multinomial[feat_name][y_val][x_val] =  float(occurences)/total
        
        def get_conditional_probability(self, x_val, y_val, feat_name):
            if type(x_val) is str:
                self.get_conditional_probability_categorical(x_val, y_val, feat_name)
            elif (type(x_val) is float) or (type(x_val) is int):
                self.get_conditional_probability_continuous(x_val, y_val, feat_name)
            return 0
        
        def get_conditional_probability_continuous(self, x_val, y_val, feat_name):
            if y_val in self.parameters_gaussian[feat_name]:
                params = self.parameters_gaussian[feat_name][y_val]
                return norm.pdf(x_val, params["mean"], params["sd"])
            return 0
        
        def get_conditional_probability_categorical(self, x_val, y_val, feat_name):
            if y_val in self.parameters_multinomial[feat_name] and x_val in self.parameters_multinomial[feat_name][y_val]:
                return self.parameters_multinomial[feat_name][y_val][x_val] 
            return 0
    
    def __init__(self):
        self.possible_classes = []
    
    def train(self, x, y):
        self.initialise_priors(y)
        self.possible_classes = y.dropna().unique()
        self.likelihood_model = self.ConditionalProbability(x, y)
    
    def initialise_priors(self, y):
        self.priors = {}
        filtered_y = y.dropna()
        for y_val in filtered_y.unique():
            occurences = filtered_y.value_counts()[y_val]
            total = len(filtered_y)
            self.priors[y_val] = float(occurences)/total

    def get_model_value(self, i, clas):
        return self.priors[clas] * np.prod([self.likelihood_model.get_conditional_probability(item[1], clas, item[0]) for item in i.items()])
            
    def predict_single(self, i):
        model_values = {clas: self.get_model_value(i, clas) for clas in self.possible_classes}
        return max(model_values, key=model_values.get)
    
    def predict(self, x):
        return x.apply(lambda i: self.predict_single(i), axis=1)
    
    

In [191]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
# and implement 90-10 splitting as specified in the project description.
def preprocess(filename):
    data = pd.read_csv(filename)
    
    # # replace ? with NAN
    # data.replace(" ?", np.nan, inplace=True)
    label_header = "label"
    x, y = data.drop(label_header, axis=1), data[label_header]
    return train_test_split(x, y, test_size=0.1, shuffle=False)
    

In [192]:
# This function should calculat prior probabilities and likelihoods (conditional probabilities) from the training data and using
# to build a naive Bayes model

def train(x, y):
    NB = NaiveBayes()
    NB.train(x, y)
    return NB

In [193]:
# This function should predict classes for new items in the testing data
def predict(model, x):
    return model.predict(x)

In [194]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels, return and output accuracy, confusion matrix and F1 score.

def evaluate():
    return

In [199]:
# This cell should act as your "main" function where you call the above functions 
# on the full ADULT data set, and print the evaluation results. [0.33 marks]
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm


# First, read in the data and apply your NB model to the ADULT data
path = "dataset/adult.csv"
train_x, test_x, train_y, test_y = preprocess(path)
# print(data.info())
# print(data.describe())
# data
# data.head(5)

model = train(train_x, train_y)
predicted = predict(model, test_x)
predicted.head(50)

# for col in data:
#     print(data[col].unique())






# # Second, print the full evaluation results from the evaluate() function






# # Third, print data statistics and model predictions, as instructed below 
# # N is the total number of instances, F the total number of attributes, L the total number of labels
# # The "class probabilities" may be unnormalized
# # The "predicted class ID" must be in range (0, L)

# print("Attribute vectors of instances [0, 1, 2]: ", ) # of the first three records in adult.csv

# print("\nNumber of instances (N): ", )
# print("Number of attributes (F): ", )
# print("Number of labels (L): ", )


# # print out the prediction results of the last three instances
# print("\n\nPredicted class log-probabilities for instance N-3: ", )
# print("Predicted class ID for instance N-3: ", )
# print("\nPredicted class log-probabilities for instance N-2: ", )
# print("Predicted class ID for instance N-2: ", )
# print("\nPredicted class log-probabilities for instance N-1: ", )
# print("Predicted class ID for instance N-1: ", )



900     <=50K
901     <=50K
902     <=50K
903     <=50K
904     <=50K
905     <=50K
906     <=50K
907     <=50K
908     <=50K
909     <=50K
910     <=50K
911     <=50K
912     <=50K
913     <=50K
914     <=50K
915     <=50K
916     <=50K
917     <=50K
918     <=50K
919     <=50K
920     <=50K
921     <=50K
922     <=50K
923     <=50K
924     <=50K
925     <=50K
926     <=50K
927     <=50K
928     <=50K
929     <=50K
930     <=50K
931     <=50K
932     <=50K
933     <=50K
934     <=50K
935     <=50K
936     <=50K
937     <=50K
938     <=50K
939     <=50K
940     <=50K
941     <=50K
942     <=50K
943     <=50K
944     <=50K
945     <=50K
946     <=50K
947     <=50K
948     <=50K
949     <=50K
dtype: object

## Part 2: Conceptual questions [8 marks for groups of 1] / [16 marks for groups of 2]


If you are in a group of 1, you should respond to Q1 and Q2.

If you are in a group of 2, you should respond to Q1, Q2, Q3 and Q4.

A response to a question should take about 100–250 words. You may need to develope codes or functions to help respond to the question here. 

#### NOTE: We strongly recommend <u>including figures or tables, etc.</u> to support your responses. The figures and tables inserted in Markdown cells must be reproducable by your code.

### Q1 [4 marks]
<u>Sensitivity</u> and <u>specificity</u> are two model evaluation metrics.  A good model should have both sensitivity and specificity high. Use the $2 \times 2$ confusion matrix returned by `evaluate()` to calculate the sensitivity and specificity. Do you see a difference between them? If so, what causes this difference? Provide suggestions to improve the model performance. 

In [4]:
# Write additional code here, if necessary (you may insert additional code cells)

Provide your text answer of 150-200 words in this cell.

### Q2 [4 marks]
You can adopt different methods for training and/or testing, which will produce different results in model evaluation. 

(a) Instead of Gaussian, <u>implement KDE</u> for  $P(X_i|c_j)$ for numeric attributes $X_i$. Compare the evaluation results with Gaussian. Which one do you think is more suitable to model $P(X_i|c_j)$, Gaussian or KDE? Observe all numeric attributes and justify your answer.

You can choose an arbitrary value for kernel bandwidth $\sigma$ for KDE, but a value between 3 and 15 is recommended. You should write code to implement KDE, not call an existing function/method such as `KernelDensity` from `scikit-learn`.

(b) Implement <u>10-fold and 2-fold cross-validations</u>.  
	Observe the evaluation results in each fold and the average accuracy, recall and specificity over all folds. 
	Comment on what is the effect by changing the values of $m$ in $m$-fold cross validation. (You can choose either Gaussian or KDE Naive Bayes.)

In [5]:
# Write additional code here, if necessary (you may insert additional code cells)

Provide your text answer of 150-200 words in this cell.

### Q3 [4 marks]
In `train()`, you are asked to treat the missing value of nominal attributes as a new category. There is another option (as suggested in Thu lecture in week 2): <u>ignoring the missing values</u>. 
Compare the two methods in both large and small datasets. Comment and explain your observations.
You can extract the first 50 records to construct a small dataset.Use Gaussian Naive Bayes only for this question.

In [6]:
# Write additional code here, if necessary (you may insert additional code cells)

Provide your text answer of 150-200 words in this cell.

### Q4 [4 marks]
In week 4, we have learned how to obtain information gain (IG) and gain ratio (GR) to choose an attribute to split a node in a decision tree. We will see how to apply them in the Naive Bayes classification.

(a) Compute the GR of each attribute $X_i$, relative to the class distribution. In the Na\"ive Bayes classifier, remove attributes in the ascending order of GR: first, remove $P(X_i|c_j)$ such that $X_i$ has the least GR; second, remove $P(X_{i'}|c_j)$ such that $X_{i'}$ has the second least GR,......, until there is only one $X_{i*}$ with the largest GR remaining in the maximand $P(c_j) P(X_{i^*} | c_j)$. Observe the <u>change of the accuracy for both Gaussian and KDE</u> (Choose bandwidth $\sigma=10$ for KDE).

(b) Compute the IG between each pair of attributes. Describe and explain your observations. Choose an attribute and implement an estimator to predict the value of `education num`. Explain why you choose this attribute. Enumerate two other examples that an attribute can be used to estimate the other and explain the reason.  

In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)

### (a)

Provide your text answer to **Question 4.a** of 100-150 words in this cell.

### (b)

Provide your text answer to **Question 4.b** of 150-200 words in this cell.

<b>Authorship Declaration</b>:

   (1) I certify that the program contained in this submission is completely
   my own individual work, except where explicitly noted by comments that
   provide details otherwise.  I understand that work that has been developed
   by another student, or by me in collaboration with other students,
   or by non-students as a result of request, solicitation, or payment,
   may not be submitted for assessment in this subject.  I understand that
   submitting for assessment work developed by or in collaboration with
   other students or non-students constitutes Academic Misconduct, and
   may be penalized by mark deductions, or by other penalties determined
   via the University of Melbourne Academic Honesty Policy, as described
   at https://academicintegrity.unimelb.edu.au.

   (2) I also certify that I have not provided a copy of this work in either
   softcopy or hardcopy or any other form to any other student, and nor will
   I do so until after the marks are released. I understand that providing
   my work to other students, regardless of my intention or any undertakings
   made to me by that other student, is also Academic Misconduct.

   (3) I further understand that providing a copy of the assignment
   specification to any form of code authoring or assignment tutoring
   service, or drawing the attention of others to such services and code
   that may have been made available via such a service, may be regarded
   as Student General Misconduct (interfering with the teaching activities
   of the University and/or inciting others to commit Academic Misconduct).
   I understand that an allegation of Student General Misconduct may arise
   regardless of whether or not I personally make use of such solutions
   or sought benefit from such actions.

   <b>Signed by</b>: [Enter your full name and student number here before submission]
   
   <b>Dated</b>: [Enter the date that you "signed" the declaration]